Quantitative Finance Project
By: Varun Gopal, Tyler Dixon, and Abhinav Kakumanu

Objective of this project is to highlight the methods learned in class and apply them with market data.

Our project revolves around examining economic data and building a predictive model that can help guide returns.

In [1]:
# Importing basic files

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime

import pandas_datareader as pdr
import pandas_datareader.famafrench
from functools import reduce


In [2]:
# Get all the datasets 
pandas_datareader.famafrench.get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
ff = pdr.get_data_famafrench('10_Industry_Portfolios', 1926)

In [4]:
type(ff)

dict

In [5]:
ff.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 'DESCR'])

In [6]:
ff.keys()
ten_ind = ff[1]

In [7]:
industry_name = [i for i in ff[0].columns]
industry_name

['NoDur',
 'Durbl',
 'Manuf',
 'Enrgy',
 'HiTec',
 'Telcm',
 'Shops',
 'Hlth ',
 'Utils',
 'Other']

In [8]:
ten_ind = ten_ind.resample('Q').apply(lambda x: (1+ (x/100)).product()-1)

In [9]:
ten_ind

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926Q3,0.062499,0.008370,0.050122,-0.014611,0.057796,0.026294,-0.000549,0.095794,0.048697,0.039668
1926Q4,0.036895,-0.054746,-0.002352,0.094341,-0.030808,0.021030,0.007802,0.077902,0.043239,-0.003773
1927Q1,-0.014018,0.012062,0.058804,-0.018129,0.070875,0.085652,0.029499,0.085339,0.067053,0.130579
1927Q2,0.060628,-0.011549,0.040557,0.029293,0.068964,-0.004983,0.034686,0.149034,0.074695,0.084061
1927Q3,0.108930,0.072260,0.101493,0.041747,0.102346,0.112249,0.131985,0.152781,0.210272,0.068413
...,...,...,...,...,...,...,...,...,...,...
2022Q1,-0.081467,-0.138146,-0.034966,0.482865,-0.126784,-0.016195,-0.093717,-0.166785,0.067288,-0.039082
2022Q2,-0.141088,-0.180388,-0.178501,-0.094793,-0.248041,-0.183191,-0.184645,-0.242632,-0.040033,-0.163498
2022Q3,-0.129309,-0.068174,-0.054447,0.037160,-0.040105,-0.142568,-0.041964,-0.013281,-0.039541,-0.035206


In [10]:
type(ten_ind.index)

pandas.core.indexes.period.PeriodIndex

We have gotten all the data from 1926 for different industries. Here are the industries:
1. Consumer Nondurables: Food, Tobacco, Textiles, Apparel, Leather, Toys
2. Consumer Durables -- Cars, TVs, Furniture, Household Appliances
3. Manufacturing -- Machinery, Trucks, Planes, Chemicals, Off Furn, Paper, Com Printing
4. Oil, Gas, and Coal Extraction and Products
5. Business Equipment -- Computers, Software, and Electronic Equipment
6. Telephone and Television Transmission
7. Wholesale, Retail, and Some Services (Laundries, Repair Shops)
8. Healthcare, Medical Equipment, and Drugs
9. Utilities
10. Other -- Mines, Constr, BldMt, Trans, Hotels, Bus Serv, Entertainment, Finance

The next step will be to gather important economic data from the Federal Reserve and map it alongside the data. 
First question is what do we want to map?

We will do our best to get variables that will not impact one specific industry. However, there may be data that skew towards a specific industry. We will highlight the bias. 

According to https://groww.in/blog/macroeconomic-factors-that-influence-us-stock-markets
1. Gross Domestic Product (GDP)
2. Inflation
3. Unemployment Rate (Payrolls)
4. Retail Sales
5. Industrial Output 

We will also get market data on interest rates, corporate profits, and corporate debt.


Get the data from Federal Reserve Economic Data (FRED)

Another Hypothesis is that markets will react to any economic indicators quickly. Since indicators such as GDP are lagging indicators, we will need market representations (such as yield spreads) that can help understand how stocks react to the propsect of economic events.

First, let us use the common models and then we can use some of the proxies later on.

In [11]:
# RGDP is real GDP... since we will examine inflation later
rgdp = pdr.get_data_fred('GDPC1', 1950)
rgdp = rgdp.pct_change().dropna()
rgdp.index = rgdp.index.to_period('Q')

In [12]:
#CPI to capture inflation
cpi = pdr.get_data_fred('CPIAUCSL',1947)
cpi = cpi.pct_change().dropna()
cpi = cpi.resample('Q').apply(lambda x: (1+ x).product()-1)
cpi.index = cpi.index.to_period('Q')

In [13]:
cpi

,CPIAUCSL
DATE,
1947Q1,0.024209
1947Q2,0.003636
1947Q3,0.034420
1947Q4,0.024956
1948Q1,0.003845
...,...
2022Q1,0.023444
2022Q2,0.025241
2022Q3,0.006145


In [14]:
# Corporate Profits
corp_profit = pdr.get_data_fred('CP', 1947)
corp_profit = corp_profit.pct_change().dropna()
corp_profit.index = corp_profit.index.to_period('Q')


In [15]:
corp_profit

,CP
DATE,
1947Q2,-0.053801
1947Q3,-0.010775
1947Q4,0.091762
1948Q1,0.057369
1948Q2,0.050634
...,...
2021Q4,-0.023466
2022Q1,0.026317
2022Q2,0.074003


In [16]:
# Unemployment Rate
unemp = pdr.get_data_fred('UNRATE', 1948)
unemp = unemp.pct_change().dropna()
unemp = unemp.resample('Q').mean()
unemp.index = unemp.index.to_period('Q')

In [17]:
unemp

,UNRATE
DATE,
1948Q1,0.085139
1948Q2,-0.032998
1948Q3,0.019231
1948Q4,0.017781
1949Q1,0.077284
...,...
2022Q1,-0.025664
2022Q2,0.000000
2022Q3,-0.008230


In [18]:
# Interest Rates (10 yr treasury rates)
irate = pdr.get_data_fred('FEDFUNDS', 1953)
irate = irate.pct_change().dropna()
irate = irate.resample('Q').apply(lambda x: (1+x).product()-1)
irate.index = irate.index.to_period('Q')

In [19]:
irate

,FEDFUNDS
DATE,
1954Q3,0.337500
1954Q4,0.196262
1955Q1,0.054688
1955Q2,0.214815
1955Q3,0.329268
...,...
2022Q1,1.500000
2022Q2,5.050000
2022Q3,1.115702


In [20]:
type(irate)

pandas.core.frame.DataFrame

We need to put the data together into a table. Additionally, we will need to group by quarters.

In [21]:
dfs = [ten_ind, rgdp, cpi, corp_profit, unemp, irate]

In [22]:
merged_df = ten_ind
for i in range(len(dfs)-1):
    if dfs[i].index[0] > dfs[i+1].index[0]:
        merged_df = merged_df.merge(dfs[i+1], how = 'left' , left_index= True, right_index = True)
    else:
        merged_df = merged_df.merge(dfs[i+1], how = 'right' , left_index= True, right_index = True)
# Since there was not as much data for Q1 2023, we excluded that from the dataset
merged_df = merged_df[:-1]

Begin running the regressions to predict the sensitivity of markets to other events.

In [23]:
# import sklearn
from sklearn.linear_model import LinearRegression

In [24]:
# Create a separate list with all the independent variables
ind_var = []
for col in merged_df.columns[10:]:
    ind_var += [col]


In [25]:
X = merged_df[ind_var]


We will now run the regression. We will use a for loop and try to store the values into a dictionary so we can then extract those values easily.

In [26]:
coef_dicts = {}
r_sqrd = {}
model = LinearRegression()



for i in industry_name:
    model.fit(X,merged_df[i])
    coef_dicts[i] = model.coef_
    r_sqrd[i] = model.score(X, merged_df[i])
       

In [27]:
r_sqrd

{'NoDur': 0.03881022597573458,
 'Durbl': 0.09597290315614193,
 'Manuf': 0.04780082543983777,
 'Enrgy': 0.09399703507956203,
 'HiTec': 0.049384070877199204,
 'Telcm': 0.03258007528747675,
 'Shops': 0.06683845906132935,
 'Hlth ': 0.055479126710092475,
 'Utils': 0.023233856014791665,
 'Other': 0.032630478974538635}

In [28]:
coef_dicts

{'NoDur': array([ 0.5437617 , -0.61248383,  0.20296422,  0.21153756, -0.04146549]),
 'Durbl': array([ 2.48000206, -1.33381449,  0.33652576,  1.00047925, -0.03882637]),
 'Manuf': array([ 1.49418249, -0.16730764,  0.2247857 ,  0.45617922, -0.03591437]),
 'Enrgy': array([ 1.30095468,  3.98047238,  0.3263599 ,  0.8658047 , -0.00593722]),
 'HiTec': array([ 1.52560358, -1.03992588,  0.30453881,  0.5626051 , -0.05594218]),
 'Telcm': array([ 0.32738296, -0.87531745,  0.22224276,  0.20398294, -0.05121019]),
 'Shops': array([ 1.1397546 , -0.85753339,  0.30241674,  0.56395411, -0.04602867]),
 'Hlth ': array([ 0.19808568, -0.97563359,  0.17707746,  0.52380209, -0.05323826]),
 'Utils': array([ 0.04841083, -0.06372742, -0.10995062,  0.02157317, -0.01327991]),
 'Other': array([ 1.14973794, -0.92195855,  0.13070961,  0.24418257, -0.03332316])}

Interpretation
Looking at the coefficients, there are some striking points to notice:

When it comes to GDP, most 60% of the industries have a positive increase in returns as GDP increases. The industries that went against that trend are Energy, Durables, Retail, and Healthcare. This means that these industries seem to do better when the GDP sees a quarterly decline.

From an inflation standpoing, an increase in inflation generally is a negative trend for 90% of ind8ustries. Energy is the sole industry that benefits quite significantly during inflation. Looking at the recent inflationary period of 2021-22 and even in the 1970's, higher energy prices were a part of large inflationary environment.

with corporate profits, most industries do not seem to have a major benefit from higher corporate profits. 90% of industries seem to increase during periods of higher corporate profit. The sole industry that bucks this trend is utilities.

On the subject of unemployment, an increase in unemployment seems to be better for all the industries. This is a very interesting dillema since conventional wisdom would suggest that unemployment rates would lead to lower returns. This will require further examination.

Looking at the interest rates, all of the industries respond negatively to higher interest rates. As we predicted, there is a higher sensitivity from the tech industry.

In [29]:
# Get data about the different industries
ind_descr = {}

for i in industry_name:
    ind_descr[i] = merged_df[i].describe()

In [30]:
# Get data about the different variables to help us with scenario planning
var_descr = {}
for i in ind_var:
    var_descr[i] = merged_df[i].describe()

In [31]:
var_descr

{'GDPC1': count    274.000000
 mean       0.007493
 std        0.011140
 min       -0.084843
 25%        0.003248
 50%        0.007515
 75%        0.011932
 max        0.078545
 Name: GDPC1, dtype: float64,
 'CPIAUCSL': count    274.000000
 mean       0.008855
 std        0.008135
 min       -0.034170
 25%        0.004776
 50%        0.007316
 75%        0.011800
 max        0.041612
 Name: CPIAUCSL, dtype: float64,
 'CP': count    274.000000
 mean       0.019965
 std        0.068414
 min       -0.411592
 25%       -0.009663
 50%        0.020859
 75%        0.049100
 max        0.500814
 Name: CP, dtype: float64,
 'UNRATE': count    274.000000
 mean       0.001436
 std        0.047651
 min       -0.102907
 25%       -0.012625
 50%       -0.004631
 75%        0.007406
 max        0.690734
 Name: UNRATE, dtype: float64,
 'FEDFUNDS': count    274.000000
 mean       0.049522
 std        0.384679
 min       -0.911602
 25%       -0.073457
 50%        0.013132
 75%        0.125000
 max       

In [32]:
var_mean_sd = {}

# Lets utilize a normal distribution to generate random economic conditions that can help us in prediction
for i in ind_var:
    var_mean_sd[i] = []
    var_mean_sd[i] += [var_descr[i]['mean']]
    var_mean_sd[i] += [var_descr[i]['std']]


In [33]:
var_mean_sd

{'GDPC1': [0.007493069929926096, 0.011139684220963451],
 'CPIAUCSL': [0.008855215215819941, 0.008135140001225708],
 'CP': [0.01996523508244142, 0.06841423231842883],
 'UNRATE': [0.0014355427526896965, 0.04765078467127316],
 'FEDFUNDS': [0.049521818295576996, 0.3846785359128082]}

We wil use random numbers using a normal distribution to help us predict returns.

In [62]:
GDP_gen = np.random.normal(var_mean_sd['GDPC1'][0], var_mean_sd['GDPC1'][1])
CPI_gen = np.random.normal(var_mean_sd['CPIAUCSL'][0], var_mean_sd['CPIAUCSL'][1])
CP_gen = np.random.normal(var_mean_sd['CP'][0], var_mean_sd['CP'][1])
UNRATE_gen = np.random.normal(var_mean_sd['UNRATE'][0], var_mean_sd['UNRATE'][1])
FedFund_gen = np.random.normal(var_mean_sd['FEDFUNDS'][0], var_mean_sd['FEDFUNDS'][1])

variable_lst = [GDP_gen,CPI_gen,CP_gen,UNRATE_gen,FedFund_gen]


# Create a prediction using the regression results above
predict = {}
for i in coef_dicts.keys():
    predict[i] = np.dot(coef_dicts[i],variable_lst)

In [63]:
print(variable_lst, '\n' , predict)

[-0.003319836421191427, 0.0105021480070953, -0.06780566644904956, -0.007996700267305257, -0.4612672805814785] 
 {'NoDur': -0.004564648012700408, 'Durbl': -0.035150671166299885, 'Manuf': -0.009041081546355478, 'Enrgy': 0.011170569448131641, 'HiTec': -0.015330353920273026, 'Telcm': -0.0033584975822851457, 'Shops': -0.01657356440260054, 'Hlth ': -0.002542337792367143, 'Utils': 0.012578357951022509, 'Other': -0.008944109879393505}
